In [1]:
import pandas as pd
import numpy as np

# Загружаем сохранённые датасеты
X_clean = pd.read_csv("X_clean.csv").values
y_clean = pd.read_csv("y_clean.csv")["label"].values  # или "class", если колонка так называлась

print("X_clean:", X_clean.shape)
print("y_clean:", y_clean.shape, "   classes:", np.unique(y_clean, return_counts=True))
from sklearn.model_selection import cross_validate, StratifiedKFold
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import numpy as np

# 5-fold стратифицированная CV и набор метрик
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scoring = {"auc": "roc_auc", "acc": "accuracy", "f1": "f1"}

# Модели: логрег (baseline), SVM (linear/rbf) и Random Forest
models = {
    "LogReg": make_pipeline(StandardScaler(), LogisticRegression(max_iter=5000)),
    "SVM-linear": make_pipeline(StandardScaler(), SVC(kernel="linear", probability=True)),
    "SVM-rbf": make_pipeline(StandardScaler(), SVC(kernel="rbf", C=1.0, gamma="scale", probability=True)),
    "RandomForest": RandomForestClassifier(n_estimators=300, random_state=42, n_jobs=-1)
}

# Оцениваем и печатаем средние метрики ± std
for name, clf in models.items():
    res = cross_validate(clf, X_clean, y_clean, cv=cv, scoring=scoring, n_jobs=-1)
    print(
        f"{name}: "
        f"AUC {res['test_auc'].mean():.3f} ± {res['test_auc'].std():.3f} | "
        f"ACC {res['test_acc'].mean():.3f} ± {res['test_acc'].std():.3f} | "
        f"F1 {res['test_f1'].mean():.3f} ± {res['test_f1'].std():.3f}"
    )


X_clean: (94, 2000)
y_clean: (94,)    classes: (array([0, 1]), array([49, 45]))
LogReg: AUC 0.987 ± 0.011 | ACC 0.936 ± 0.021 | F1 0.933 ± 0.016
SVM-linear: AUC 0.998 ± 0.004 | ACC 0.926 ± 0.042 | F1 0.924 ± 0.034
SVM-rbf: AUC 0.969 ± 0.026 | ACC 0.915 ± 0.042 | F1 0.908 ± 0.047
RandomForest: AUC 0.950 ± 0.021 | ACC 0.926 ± 0.042 | F1 0.919 ± 0.048


как мы можем видеть, наибольший ROC AUC имеет  SVM-linear